## Application of BPR on Movielens

In [1]:
#%load_ext watermark
%load_ext autoreload 
%autoreload 2

In [2]:
import os
import sys
import itertools
import numpy as np
import pandas as pd
from subprocess import call
from pybpr import *
import matplotlib.pyplot as plt
from functools import partial

In [3]:
%%time
#df = load_movielens_data('ml-1m')
df = load_movielens_data('ml-100k')
dfshort = df[df['rating'] > 0]
df.head()

CPU times: user 22 ms, sys: 9 ms, total: 31 ms
Wall time: 124 ms


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
cf = UserItemInteractions(
    name='MovieLens-100k',
    users=dfshort['user_id'],
    items=dfshort['item_id'],
    min_num_rating_per_user=10,
    min_num_rating_per_item=10,
    num_cores = 1
)
cf.print_memory_usage()
cf.generate_train_test(user_test_ratio=0.2)

--- Memory usage for MovieLens-100k:
Sparse User-Item matrix = 0.09 MB
User-Item dataframe df = 1.98 MB
Item dataframe df_item = 0.1 MB
Item dataframe df_user = 0.05 MB
---


In [5]:
bpr = BPR(
    num_features=10,
    reg_lambda=0.,
    num_iters=50,
    learning_rate = 0.5,
    batch_size=5,
    initial_std=0.01,
    seed=None
)
bpr_ndcg_func = partial(
    cf.get_ndcg_metric,
    num_items=10
)
bpr.fit(cf.R_train, bpr_ndcg_func)
ndcg_df = pd.DataFrame(bpr.ndcg_metric)

BPR: 100%|████████████████████████████████████████████████| 50/50 [00:36<00:00,  1.35it/s]


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,3))
ax[0].plot(ndcg_df['test'])
ax[1].plot(ndcg_df['train'])

In [ ]:
#bpr_ndcg_func(test=True), bpr_ndcg_func(test=False)

In [ ]:
ndcg_df